## Google colab setup

In [1]:
try:
  import google.colab
  IS_COLAB = True
except:
  IS_COLAB = False

In [2]:
if IS_COLAB:
  !pip install tensorflow_addons
  !pip install unidecode
  !pip install transformers
  # !pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 37.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 9.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp /content/drive/MyDrive/SemiSupervised/*.csv .

## Imports and setup

In [5]:
%%capture
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import re
import time
import spacy
import numpy as np
from transformers import AutoTokenizer, TFAutoModel
from unidecode import unidecode
# from utils import prepare_train_ds, prepare_ds

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

## Prepare data splits

In [6]:
data = pd.read_csv("train.csv")

In [7]:
def preprocess(x):
    s = unidecode(x)
    s = str.lower(s)
    s = re.sub(r"\[[a-z]+\]","", s)
    s = re.sub(r"\*","", s)
    s = re.sub(r"[^a-zA-Z0-9]+"," ",s)
    s = re.sub(r" +"," ",s)
    s = re.sub(r"(.)\1+",r"\1",s)

    return s

In [8]:
label_ids = {label_name:i for i, label_name in enumerate(sorted(set(data["label"])))}
label_ids

{'ABUSE': 0, 'INSULT': 1, 'OTHER': 2, 'PROFANITY': 3}

In [9]:
tok_robert = AutoTokenizer.from_pretrained("readerbench/RoBERT-base")

In [ ]:
def split_ssl_data(ids_array,mask_array,labels,num_classes,label_percent):
  labeled = None
  unlabeled = None

  for class_idx in range(num_classes):
    class_ids = ids_array[labels==class_idx]
    class_mask = mask_array[labels==class_idx]
    sz = int(label_percent * class_ids.shape[0])

    labels_reduced = labels[labels==class_idx][:sz]
    labeled_ids, unlabeled_ids = class_ids[:sz], class_ids[sz:]
    labeled_mask, unlabeled_mask = class_mask[:sz], class_mask[sz:]

    if not labeled:
      labeled = (labeled_ids, labeled_mask, labels_reduced)
      unlabeled = (unlabeled_ids, unlabeled_mask)
    else:
      labeled = (
          np.concatenate([labeled[0],labeled_ids]),
          np.concatenate([labeled[1],labeled_mask]),
          np.concatenate([labeled[2],labels_reduced])
      )
      unlabeled = (
          np.concatenate([unlabeled[0],unlabeled_ids]),
          np.concatenate([unlabeled[1],unlabeled_mask]),
      )

  return labeled, unlabeled

In [20]:
def preprocess_robert(x):
  t = tok_robert(x,padding="max_length",max_length=96,truncation=True,return_tensors='np')
  return t["input_ids"], t["attention_mask"]

def map_func(input_ids, masks, labels):
  return {'input_ids': input_ids, 'attention_mask': masks}, labels

def map_func2(input_ids, masks):
  return {'input_ids': input_ids, 'attention_mask': masks}

def prepare_ds2(filename,batch_size=64,repeat=False,f=False):
  df = pd.read_csv(filename)
  texts = list(df['comment'].map(preprocess))

  ids_array = None
  mask_array = None
  for idx in range(0,len(texts),1000):
      batch_texts = texts[idx:idx+1000]
      inputs = tok_robert(batch_texts,padding="max_length",max_length=96,truncation=True,return_tensors='np')
      if idx == 0:
        ids_array = inputs["input_ids"]
        mask_array = inputs["attention_mask"]
      else:
        ids_array = np.concatenate([ids_array,inputs["input_ids"]],axis=0)
        mask_array = np.concatenate([mask_array,inputs["attention_mask"]],axis=0)

  res_ds = tf.data.Dataset.from_tensor_slices((ids_array, mask_array)).map(map_func2).shuffle(len(df)).batch(batch_size)
  if repeat:
    res_ds = res_ds.repeat()
  return res_ds


def prepare_ds(filename,batch_size=64,repeat=False):
  df = pd.read_csv(filename)
  X_id_mask = df['text'].map(preprocess).apply(preprocess_robert).apply(pd.Series)

  X_id_mask.columns = ["input_ids","attention_mask"]

  ids_array = np.squeeze(np.stack(X_id_mask.input_ids.values), axis=1)
  mask_array = np.squeeze(np.stack(X_id_mask.attention_mask.values), axis=1)

  labels = df["label"].map(lambda x: label_ids[x]).values

  res_ds = tf.data.Dataset.from_tensor_slices((ids_array, mask_array, labels)).map(map_func).shuffle(len(df)).batch(batch_size)
  if repeat:
    res_ds = res_ds.repeat()
  return res_ds

In [29]:
labeled_ds = prepare_ds("train.csv",repeat=True,batch_size=16)
unlabeled = prepare_ds2("big_unlabeled.csv",repeat=True,f=True,batch_size=256)
test_ds = prepare_ds("test.csv")
val_ds = prepare_ds("validation_internal.csv")

## Model definition and declaration

In [22]:
class FixMatchTune(tf.keras.Model):
  def __init__(
      self,
      encoder_name="readerbench/RoBERT-base",
      num_classes=4,
      **kwargs
  ):
    super(FixMatchTune,self).__init__(**kwargs)

    self.bert = TFAutoModel.from_pretrained(encoder_name)
    self.num_classes = num_classes
    self.weak_augment = tf.keras.layers.GaussianNoise(stddev=0.5)
    self.strong_augment = tf.keras.layers.GaussianNoise(stddev=5)

    self.cls_head = tf.keras.Sequential([
      tf.keras.layers.Dense(256,activation="relu"),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(64,activation="relu"),
      tf.keras.layers.Dense(self.num_classes, activation="softmax")
    ])

  def call(self, inputs, training):
    ids, mask = inputs
    
    embeds = self.bert(input_ids=ids, attention_mask=mask,training=training).pooler_output

    strongs = self.strong_augment(embeds,training=training)
    weaks = self.weak_augment(embeds,training=training)

    strong_preds = self.cls_head(strongs,training=training)
    weak_preds = self.cls_head(weaks,training=training)

    return weak_preds, strong_preds


In [30]:
# model = FreeMatch()
model = FixMatchTune()

## Train and evaluate

In [31]:
optim = tfa.optimizers.AdamW(weight_decay=0.001,learning_rate=0.005)
optim2 = tfa.optimizers.AdamW(weight_decay=0.0,learning_rate=0.00001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_prec_metric = tf.keras.metrics.Precision(name="precision")
val_recall_metric = tf.keras.metrics.Recall(name="recall")
f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')

In [32]:
thresh = 0.8
unsup_weight = 1.0
steps_per_epoch = 600

In [33]:
@tf.function
def train_step_tune(x,y,xu):

  with tf.GradientTape() as tape:
      wl,_ = model([x["input_ids"],x["attention_mask"]],training=True)
      wu,su = model([xu["input_ids"],xu["attention_mask"]],training=True)
      ls = loss_fn(y, wl)
      mask = tf.reduce_max(wu,axis=1)>thresh
      wu = wu[mask]
      su = su[mask]
      lu = loss_fn(tf.argmax(wu,axis=1),su)

      loss = ls + unsup_weight * lu

  grads = tape.gradient(loss, [model.cls_head.trainable_weights, model.bert.trainable_weights])
  optim.apply_gradients(zip(grads[0], model.cls_head.trainable_weights))
  optim2.apply_gradients(zip(grads[1], model.bert.trainable_weights))

  return loss

@tf.function
def train_step(x,y,xu):

  with tf.GradientTape() as tape:
      wl,_ = model([x["input_ids"],x["attention_mask"]],training=True)
      wu,su = model([xu["input_ids"],xu["attention_mask"]],training=True)
      ls = loss_fn(y, wl)
      mask = tf.reduce_max(wu,axis=1)>thresh
      wu = wu[mask]
      su = su[mask]
      lu = loss_fn(tf.argmax(wu,axis=1),su)

      loss = ls + unsup_weight * lu

  grads = tape.gradient(loss, model.trainable_weights)
  optim.apply_gradients(zip(grads, model.trainable_weights))

  return loss

@tf.function
def test_step(x, y):
    wpred,_ = model([x["input_ids"],x["attention_mask"]], training=False)
    val_acc_metric.update_state(y, wpred)
    true_hot = tf.one_hot(y, 4)
    val_prec_metric.update_state(true_hot, wpred)
    val_recall_metric.update_state(true_hot, wpred)
    f1_metric_micro.update_state(true_hot, wpred)
    f1_metric_macro.update_state(true_hot, wpred)

In [34]:
EPOCHS=4
max_val_acc = 0.0
best_weights = None

l_iter = iter(labeled_ds)
u_iter = iter(unlabeled)
for epoch in range(EPOCHS):
  print(f"Epoch {epoch}")
  start_time = time.time()

  for step in range(steps_per_epoch):
    x, y = next(l_iter)
    xu = next(u_iter)
    
    loss = train_step_tune(x,y,xu)
    if step % 50 == 0:
      print(
        "Training loss (for one batch) at step %d: %.4f"
        % (step, float(loss))
      )
  for x_batch_val, y_batch_val in val_ds:
      test_step(x_batch_val, y_batch_val)

  acc = float(val_acc_metric.result())
  prec = float(val_prec_metric.result())
  recall = float(val_recall_metric.result())
  micro = float(f1_metric_micro.result())
  macro = float(f1_metric_macro.result())

  val_acc_metric.reset_states()
  val_prec_metric.reset_states()
  val_recall_metric.reset_states()
  f1_metric_micro.reset_states()
  f1_metric_macro.reset_states()

  if acc > max_val_acc:
    max_val_acc = acc
    best_weights = model.get_weights()
  print(f"Validation acc: {acc:.3f} precision: {prec:.3f} recall: {recall:.3f} f1_micro: {micro:.3f} f1_macro: {macro:.3f}")
  print("Time taken: %.2fs" % (time.time() - start_time))
print("Restoring best weights relative to validation accuracy...")
model.set_weights(best_weights)

Epoch 0
Training loss (for one batch) at step 0: 2.5836
Training loss (for one batch) at step 50: 1.4553
Training loss (for one batch) at step 100: 1.3861
Training loss (for one batch) at step 150: 1.3439
Training loss (for one batch) at step 200: 1.4262
Training loss (for one batch) at step 250: 0.9152
Training loss (for one batch) at step 300: 1.3448
Training loss (for one batch) at step 350: 1.0723
Training loss (for one batch) at step 400: 1.1227
Training loss (for one batch) at step 450: 0.7560
Training loss (for one batch) at step 500: 1.1155
Training loss (for one batch) at step 550: 0.6989
Validation acc: 0.737 precision: 0.815 recall: 0.627 f1_micro: 0.709 f1_macro: 0.591
Time taken: 301.44s
Epoch 1
Training loss (for one batch) at step 0: 0.6670
Training loss (for one batch) at step 50: 0.9217
Training loss (for one batch) at step 100: 0.4628
Training loss (for one batch) at step 150: 0.6175
Training loss (for one batch) at step 200: 1.1097
Training loss (for one batch) at st

In [35]:
for x_batch_val, y_batch_val in test_ds:
    test_step(x_batch_val, y_batch_val)
acc = float(val_acc_metric.result())
prec = float(val_prec_metric.result())
recall = float(val_recall_metric.result())
micro = float(f1_metric_micro.result())
macro = float(f1_metric_macro.result())

val_acc_metric.reset_states()
val_prec_metric.reset_states()
val_recall_metric.reset_states()
f1_metric_micro.reset_states()
f1_metric_macro.reset_states()
print(f"Test acc: {acc:.3f} precision: {prec:.3f} recall: {recall:.3f} f1_micro: {micro:.3f} f1_macro: {macro:.3f}")

Test acc: 0.814 precision: 0.832 recall: 0.793 f1_micro: 0.812 f1_macro: 0.801


In [ ]:
model.cls_head.save_weights("./checkpoints/freematch_notune")